# Recurrent Neural Network

In [1]:
import pandas as pd
# Successfully installed pandas-1.3.3
import numpy as np
# Successfully installed fsspec-2021.8.1

In [2]:
import gensim.downloader as api
pretrained_model = api.load('word2vec-google-news-300')

In [3]:
# setting the vocab and taking its length
pretrained_vocab = set(pretrained_model.key_to_index)
len(pretrained_vocab)

3000000

In [4]:
import gensim
print(gensim.__version__)

4.1.2


In [5]:
my_model_binary = gensim.models.Word2Vec.load("./my_model_binary_anlp_assignment2.model")
# setting the vocab and taking its length
mymodel_vocab_binary = set(my_model_binary.wv.key_to_index)
len(mymodel_vocab_binary)

12989

In [6]:
my_model_ternary = gensim.models.Word2Vec.load("./my_model_ternary_anlp_assignment2.model")
# setting the vocab and taking its length
mymodel_vocab_ternary = set(my_model_ternary.wv.key_to_index)
len(mymodel_vocab_ternary)

14255

In [7]:
def remove_words(review , vocab):
    nn =[]
    for i in review:
        if i in vocab:
            nn.append(i)
    return nn

In [8]:
# creating the word embedding with limit of 50
def adjust_len(review):
    nn =[]
    relen = len(review)
    if relen > 20 :
        for words in review[0:20]:
            nn.append(words)
    elif relen < 20 :
        for words in review:
            nn.append(words)
        acc = 20 - relen
        for i in range(acc):
            nn.append(0)
    elif relen == 20:
        for words in review:
            nn.append(words)
    return nn

In [25]:
def embedded_vectors_fifty(sentences , vocab , model):
    vectors_data=[]
    for review in sentences:
        vectors=[]
        for words in review:
            if words in vocab:
                vectors.append(model[words])
            else:
                vectors.append(np.zeros(300))
#         vectors = map(float , (vectors))
        vectors_data.append(vectors)
    return vectors_data 

In [10]:
def embedded_vectors_mymodel_fifty(sentences , vocab , model):
    vectors_data=[]
    for review in sentences:
        vectors=[]
        for words in review:
            if words in vocab:
                vectors.append(model.wv[words])
            else:
                vectors.append(np.zeros(300))
        vectors_data.append(vectors)
    return vectors_data 

In [10]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim 
import torch.nn.functional as F

# Part A

# Binary Classification

In [11]:
class dds(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __getitem__(self , index):
        return self.x[index] , self.y[index]
    
    def __len__(self):
        return len(self.x)
    

In [12]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        # batch_dim = number of samples per batch
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, hn = self.rnn(x, h0.detach())
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [13]:
input_dim = 300 
hidden_dim=50
layer_dim=1
output_dim =2 
model = RNN(input_dim, hidden_dim, layer_dim, output_dim)

In [14]:

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
print(model)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

RNN(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)


# Pretrained Model

In [20]:
binary = pd.read_csv("binary_dataset_labels01.csv")
from ast import literal_eval
binary['Review'] = binary['Review'].apply(literal_eval)
binary['Review'] = binary['Review'].apply(lambda x : remove_words(x ,pretrained_vocab ))
binary = binary[binary['Review'].apply(lambda x : len(x) > 0)]
binary['Review'] = binary['Review'].apply(lambda x : adjust_len(x))
sent = binary['Review']



In [26]:
vector_pmb = embedded_vectors_fifty(sent, pretrained_vocab, pretrained_model)
from sklearn.model_selection import train_test_split

In [27]:
X_train_pmb, X_test_pmb, y_train_pmb, y_test_pmb = train_test_split(vector_pmb, binary['Label'], random_state=42, test_size=0.2)

In [28]:
vector_pmb[0][0]

array([ 7.17773438e-02,  2.08007812e-01, -2.84423828e-02,  1.78710938e-01,
        1.32812500e-01, -9.96093750e-02,  9.61914062e-02, -1.16699219e-01,
       -8.54492188e-03,  1.48437500e-01, -3.34472656e-02, -1.85546875e-01,
        4.10156250e-02, -8.98437500e-02,  2.17285156e-02,  6.93359375e-02,
        1.80664062e-01,  2.22656250e-01, -1.00585938e-01, -6.93359375e-02,
        1.04427338e-04,  1.60156250e-01,  4.07714844e-02,  7.37304688e-02,
        1.53320312e-01,  6.78710938e-02, -1.03027344e-01,  4.17480469e-02,
        4.27246094e-02, -1.10351562e-01, -6.68945312e-02,  4.19921875e-02,
        2.50000000e-01,  2.12890625e-01,  1.59179688e-01,  1.44653320e-02,
       -4.88281250e-02,  1.39770508e-02,  3.55529785e-03,  2.09960938e-01,
        1.52343750e-01, -7.32421875e-02,  2.16796875e-01, -5.76171875e-02,
       -2.84423828e-02, -3.60107422e-03,  1.52343750e-01, -2.63671875e-02,
        2.13623047e-02, -1.51367188e-01,  1.04003906e-01,  3.18359375e-01,
       -1.85546875e-01,  

In [29]:
X_train_pmb[0][0]

array([-0.00187683,  0.20214844,  0.15136719, -0.16503906,  0.03149414,
        0.07421875,  0.16503906, -0.1953125 ,  0.28125   ,  0.19628906,
       -0.23925781, -0.08398438, -0.04370117,  0.00291443,  0.00668335,
        0.16113281,  0.19335938,  0.07128906,  0.12353516, -0.20507812,
        0.30078125,  0.171875  ,  0.44726562, -0.08105469,  0.36132812,
        0.12158203, -0.08251953,  0.02612305,  0.30664062, -0.13085938,
       -0.0402832 ,  0.07910156, -0.09375   , -0.11328125,  0.16601562,
       -0.09619141,  0.21191406,  0.19042969, -0.01318359,  0.21972656,
        0.18945312, -0.06347656,  0.34375   , -0.3125    ,  0.24414062,
       -0.00631714, -0.05273438,  0.09326172, -0.11474609, -0.04394531,
       -0.140625  ,  0.05444336,  0.16308594, -0.00312805,  0.19140625,
        0.21289062, -0.16503906, -0.06298828,  0.14257812, -0.296875  ,
       -0.27929688,  0.31054688, -0.16503906,  0.11621094,  0.0703125 ,
       -0.08056641,  0.06787109,  0.00738525, -0.11669922, -0.10

In [20]:
# print(X_train_pmb[3][0])

In [21]:
# X_test_pmb[3]

In [30]:
X_train_pmb, X_test_pmb = map(torch.tensor , (X_train_pmb, X_test_pmb))

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 7673184000 bytes.

In [23]:
X_train_pmb = X_train_pmb.float()
X_test_pmb = X_test_pmb.float()

In [24]:
# X_train_pmb = torch.tensor(X_train_pmb,dtype=torch.long)
# X_test_pmb = torch.tensor(X_test_pmb,dtype=torch.long)

In [25]:
X_train_pmb.dtype

torch.float64

In [26]:
y_train_pmb = torch.tensor(y_train_pmb.values)
y_test_pmb = torch.tensor(y_test_pmb.values)

In [27]:
mydata_train_pmb = dds(X_train_pmb,y_train_pmb)
mydata_test_pmb = dds(X_test_pmb , y_test_pmb)

In [28]:
train_loader_pmb = DataLoader( dataset = mydata_train_pmb)
test_loader_pmb = DataLoader(dataset = mydata_test_pmb)

In [31]:
# 
epochs = 10
model = model.double()
for epoch in range(epochs):
    for features , targets in train_loader_pmb:
#         features = features.double()
#         print(features)
        output = model.forward(features)
#         print(output.size())
#         print(output)
#         print(targets.squeeze(1).size())
        loss = criterion(output , targets.squeeze(1).long().view(1,-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    
    c = torch.argmax(output.data ,dim =1)
    train_accuracy = (c == targets).sum().item()/targets.shape[0]
    #print('Epoch Number {0} accuracy'.format(epoch) , train_accuracy)
    for featuress , targetss in test_loader_pmb:
        predict =model.forward(featuress)
        
    c1 = torch.argmax(predict.data ,dim =1)
    test_accuracy = (c1 == targetss).sum().item()/targetss.shape[0]
    print("Epoch {}/{} , Loss = {:.4f} , Train accuracy = {:.4f} , Test accuracy = {:.4f} ".format(
    epoch,epochs, loss, train_accuracy, test_accuracy 
    ))
    model.train()

NameError: name 'train_loader_pmb' is not defined

# Binary Model

# Ternary Classification

# Pretrained Model

# Ternary Model

# Part B

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(GRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        # batch_dim = number of samples per batch
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        # (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.gru(x, h0.detach()) 
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
input_dim = 300 
hidden_dim=50
layer_dim=1
output_dim =2 
model3 = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

# Binary Classification

# Pretrained Model

# Binary Model

# Ternary Classification

In [ ]:
input_dim = 300 
hidden_dim=50
layer_dim=1
output_dim =3
model3 = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

# Pretrained Model

# Ternary Model